# 实验一
实现ResNet网络子模块，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [21]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, AveragePooling2D,Input,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Concatenate
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

### 定义Inception网络结构



In [33]:
img_width, img_height = 50, 50

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
    bn_axis = 1
else:
    input_shape = (img_width, img_height, 3)
    bn_axis = 3

input_img = Input (shape = input_shape)    

#ResNet Module
x = layers.Conv2D(64, (1, 1), padding='same', kernel_initializer='he_normal')(input_img)
x = layers.BatchNormalization(axis=bn_axis)(x)
x = layers.Activation('relu')(x)

x = layers.Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization(axis=bn_axis)(x)
x = layers.Activation('relu')(x)

x = layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_normal'')(x)
x = layers.BatchNormalization(axis=bn_axis)(x)

x = layers.add([x, input_img])
x = layers.Activation('relu')(x)

#fully connected layer
out = Flatten()(out)
out = Dense(48, activation='relu')(out)
                  
#output layer
out = Dense(2, activation='softmax')(out)

model = Model(x, out)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 查看model架构



In [34]:
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 50, 50, 3)    0                                            
__________________________________________________________________________________________________
conv2d_109 (Conv2D)             (None, 50, 50, 16)   48          input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_111 (Conv2D)             (None, 50, 50, 16)   48          input_18[0][0]                   
__________________________________________________________________________________________________
batch_normalization_102 (BatchN (None, 50, 50, 16)   64          conv2d_109[0][0]                 
___________________________________________________________________________________________

### 定义ImageDataGenerator


In [27]:
train_data_dir = r'C:\Users\coffe\Desktop\dogs-vs-cats\train'
validation_data_dir = r'C:\Users\coffe\Desktop\dogs-vs-cats\validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 5


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [35]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

ResourceExhaustedError: OOM when allocating tensor with shape[2340000,48] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node dense_29/random_uniform/RandomUniform (defined at C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4357) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'dense_29/random_uniform/RandomUniform':
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Progra

### 使用训练后模型预测图像





In [48]:
import cv2
img = cv2.resize(cv2.imread(r'C:\Users\coffe\Desktop\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = model.predict(x)


print(score)

[[0.99895906]]
